stock news reaction period is generally 2 days, therefore base predictions off sentiment of articles from last 2 days
try stocknews for provided sentiment analysis of headlines
finvizfinance for stock news articles to analyze
try huggingface for pretrained financial news sentiment models
use kaggle data to train a new financial news sentiment model
use yfinance for market data

# Sentiment Analysis of Financial News

In [4]:
import numpy as np
import pandas as pd
import pickle as pk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
import torch
from torch.utils.data.dataset import Dataset
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


In [19]:
df = pd.read_csv("../data/data.csv")


#downsize data for testing, use full data for model training
df = df.head(500)

df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [13]:
tokenizer = get_tokenizer('basic_english')
df["Sentence"] = df["Sentence"].apply(tokenizer)

def tokenGen(data_iter):
    for i in data_iter:
        yield tokenizer(i)

vocab = build_vocab_from_iterator(
    df["Sentence"],

)

vocab.get_itos()


['.',
 'the',
 ',',
 'of',
 'in',
 'and',
 'to',
 'a',
 "'",
 'for',
 's',
 'eur',
 'from',
 'company',
 'is',
 'on',
 'will',
 'sales',
 'by',
 'at',
 'its',
 'with',
 'has',
 'be',
 'as',
 '%',
 'mn',
 'finnish',
 '-',
 '1',
 'profit',
 'said',
 ')',
 '2',
 'net',
 '(',
 'it',
 'was',
 'million',
 '4',
 'are',
 '7',
 'group',
 '5',
 'mln',
 '3',
 'share',
 'that',
 'up',
 'year',
 'operating',
 '2008',
 'have',
 '2009',
 '0',
 '6',
 'down',
 'm',
 'quarter',
 '2010',
 '8',
 'period',
 'finland',
 'shares',
 'which',
 '000',
 'an',
 'new',
 '10',
 'also',
 'market',
 'price',
 '9',
 '$',
 'been',
 '2007',
 'loss',
 'stock',
 'business',
 'euro',
 'http',
 'oyj',
 'pct',
 'this',
 'total',
 '//stks',
 'about',
 'or',
 'solutions',
 'today',
 '2006',
 '30',
 'building',
 'first',
 'helsinki',
 'nokia',
 'not',
 'some',
 '13',
 '``',
 'financial',
 'out',
 'under',
 'all',
 'corresponding',
 'deal',
 'decreased',
 'earlier',
 'into',
 'over',
 'per',
 'prices',
 'services',
 'two',
 '//t

In [21]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["Sentence"])
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [72]:
tensor = torch.tensor(df["Sentence"])

ValueError: too many dimensions 'str'

In [ ]:
class financeTextDataset(Dataset):
    def __init__(self, dataframe):
        self.wordEmbeddings = CountVectorizer().fit_transform(dataframe["Sentence"])
        self.labels = dataframe["Sentiment"]
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.wordEmbeddings[index], self.labels[index]